In [8]:
!pip install sk-video

In [0]:
import skvideo.io  
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.cluster import KMeans

In [10]:
video = skvideo.io.vread("/content/videoplayback.mp4")  
print(video.shape)

(1682, 360, 640, 3)


In [0]:
colors = []
for z in range(0, video.shape[0]):
  colors.append(video[z][0][0])

In [0]:
pixel = pd.DataFrame(colors, columns=["R","G","B"])

In [0]:
km = KMeans(n_clusters = 2)
km = km.fit(pixel)

In [0]:
pixel["label"] = km.labels_

In [16]:
pixel

,R,G,B,label
0,101,109,118,0
1,101,109,118,0
2,101,109,118,0
3,101,109,118,0
4,101,109,118,0
...,...,...,...,...
1677,101,110,123,0
1678,101,110,123,0
1679,101,110,123,0
1680,101,110,123,0


In [17]:
fig = px.scatter_3d(pixel, x = "R", y = "G", z = "B", color="label")
fig.show()